# 基礎班時問 1330

問1. 怎麼用python抓share point的資料

程式相關問題通常把問題翻譯成英文的簡短句子丟到 google 裡, 第一筆 stack overflow 的資料有 5, 6 成是解答

https://stackoverflow.com/questions/53671547/python-download-files-from-sharepoint-site

# 影像班時問 1430

## yolo 模型使用

In [ ]:
from

## yolo 模型訓練步驟
- 準備資料
- 選擇模型
- 訓練
- 使用模型


# 綜合時間 1530